### Imports

In [75]:
from mpl_toolkits import mplot3d
from sklearn.metrics import silhouette_score
from sklearn.model_selection import RandomizedSearchCV
from tqdm import tqdm
from utils.utils import *

import hdbscan
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import os
import pandas as pd
import sys

%load_ext autotime

The autotime extension is already loaded. To reload it, use:
  %reload_ext autotime
time: 1.58 ms (started: 2023-05-27 18:16:52 -05:00)


### Parameters

In [76]:
import json
  
# Opening JSON file
file = open("parameters.json")
params = json.load(file)

dir = os.path.dirname(os.getcwd())
print(dir)

method = params["method"][params["method_idx"]]
print("Method:\t\t", method)

group = params["group"][params["group_idx"]]
print("Group:\t\t", group)

subgroups = [str(k + 1) for k in range(group[1])]
print("Subgroup:\t", subgroups)

dimensions = params["dimensions"]
dimension = dimensions[0]
print("Dimensions:\t", dimensions)

/home/ealvarez/Project/GNN_Filter
Method:		 dgi
Group:		 ['WT', 5]
Subgroup:	 ['1', '2', '3', '4', '5']
Dimensions:	 [3]
time: 3.61 ms (started: 2023-05-27 18:16:52 -05:00)


### Get common subgraphs

In [77]:
list_graphs = []

time: 611 µs (started: 2023-05-27 18:16:52 -05:00)


In [78]:
graphs = []
for item in tqdm(subgroups):
    weighted_edges = pd.read_csv("{}/output_preprocessing/edges/{}_edges_{}.csv".format(dir, group[0], item))
    G = nx.from_pandas_edgelist(weighted_edges, "source", "target", edge_attr="weight")
    graph_detail(G)
    graphs.append(G)

 20%|███████████▏                                            | 1/5 [00:02<00:10,  2.58s/it]

Num. nodes: 6234
Num. edges: 1243057



 40%|██████████████████████▍                                 | 2/5 [00:04<00:06,  2.20s/it]

Num. nodes: 6231
Num. edges: 939038



 60%|█████████████████████████████████▌                      | 3/5 [00:09<00:06,  3.37s/it]

Num. nodes: 6233
Num. edges: 2252847



 80%|████████████████████████████████████████████▊           | 4/5 [00:17<00:05,  5.18s/it]

Num. nodes: 6226
Num. edges: 3728175



100%|████████████████████████████████████████████████████████| 5/5 [00:25<00:00,  5.20s/it]

Num. nodes: 6226
Num. edges: 4071713

time: 26 s (started: 2023-05-27 18:16:52 -05:00)


In [79]:
H1 = get_subgraphs(graphs)
graph_detail(H1)

list_graphs.append(H1.copy())

100%|████████████████████████████████████████████████████████| 4/4 [00:34<00:00,  8.73s/it]


Num. nodes: 4202
Num. edges: 135076

time: 41 s (started: 2023-05-27 18:17:19 -05:00)


In [80]:
df_edge_embeddings_join_filter_count = pd.DataFrame(H1.edges())
df_edge_embeddings_join_filter_count.columns = ["source", "target"]
df_edge_embeddings_join_filter_count

,source,target
0,243.0404,267.1048
1,243.0404,401.2187
2,243.0404,661.3471
3,243.0404,267.0965
4,243.0404,416.1414
...,...,...
135071,579.4245,579.4980
135072,663.3295,663.3449
135073,226.0573,226.0718
135074,412.8122,412.8271


time: 91.7 ms (started: 2023-05-27 18:18:00 -05:00)


In [81]:
# Get weight
df_edge_embeddings_join_filter_count_weight = df_edge_embeddings_join_filter_count.copy()
s = []
t = []
for row in df_edge_embeddings_join_filter_count_weight.itertuples():
    if row[1] > row[2]:
        s.append(row[2])
        t.append(row[1])
    else:
        s.append(row[1])
        t.append(row[2])
df_edge_embeddings_join_filter_count_weight["source"] = s
df_edge_embeddings_join_filter_count_weight["target"] = t

# df_edge_embeddings_join_filter_count_weight = df_edge_embeddings_join_filter_count.copy()
df_edge_embeddings_join_filter_count_weight.sort_values(["source", "target"], ascending=True, inplace=True)
df_edge_embeddings_join_filter_count_weight["idx"] = df_edge_embeddings_join_filter_count_weight["source"].astype(str) + "-" + df_edge_embeddings_join_filter_count_weight["target"].astype(str)
list_aux = df_edge_embeddings_join_filter_count_weight.iloc[:, -1].values

for i in tqdm(subgroups):
    df_edges = pd.read_csv("{}/output_preprocessing/edges/{}_edges_{}.csv".format(dir, group[0], i))
    df_edges.sort_values(["source", "target"], ascending=True, inplace=True)
    df_edges["idx"] = df_edges["source"].astype(str) + "-" + df_edges["target"].astype(str)
    
    filter = df_edges["idx"].isin(list_aux)
    temp = df_edges[filter]
    list_temp = temp.iloc[:, -2].values
    df_edge_embeddings_join_filter_count_weight["subgroup{}".format(i)] = list_temp
df_edge_embeddings_join_filter_count_weight


100%|████████████████████████████████████████████████████████| 5/5 [00:16<00:00,  3.33s/it]


,source,target,idx,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5
133290,59.0137,59.0291,59.0137-59.0291,0.954735,0.928018,0.841093,0.601605,0.970185
131497,59.0137,349.2758,59.0137-349.2758,0.796859,0.788310,0.560861,0.551982,0.807207
110528,59.0137,350.2793,59.0137-350.2793,0.807013,0.779306,0.576143,0.563560,0.808223
132969,59.0137,350.3033,59.0137-350.3033,0.854806,0.751778,0.539172,0.650983,0.883973
129629,59.0137,389.2739,59.0137-389.2739,0.823682,0.886260,0.723124,0.515230,0.917952
...,...,...,...,...,...,...,...,...
130790,980.7146,981.7177,980.7146-981.7177,0.743341,0.954130,0.878971,0.816034,0.817595
108711,981.5305,981.7177,981.5305-981.7177,0.571209,0.850899,0.670445,0.544705,0.731794
131593,981.7177,996.7096,981.7177-996.7096,0.557537,0.784976,0.563671,0.732291,0.554250
134700,985.5091,985.5490,985.5091-985.549,0.798737,0.577365,0.723628,0.875572,0.644783


time: 16.9 s (started: 2023-05-27 18:18:00 -05:00)


In [82]:
# Dispersion (std)
df_edge_embeddings_join_filter_count_weight_std = df_edge_embeddings_join_filter_count_weight.copy()
df_edge_embeddings_join_filter_count_weight_std["std"] = np.std(df_edge_embeddings_join_filter_count_weight_std.iloc[:, -len(subgroups):], axis=1)
df_edge_embeddings_join_filter_count_weight_std

,source,target,idx,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5,std
133290,59.0137,59.0291,59.0137-59.0291,0.954735,0.928018,0.841093,0.601605,0.970185,0.136282
131497,59.0137,349.2758,59.0137-349.2758,0.796859,0.788310,0.560861,0.551982,0.807207,0.118268
110528,59.0137,350.2793,59.0137-350.2793,0.807013,0.779306,0.576143,0.563560,0.808223,0.112406
132969,59.0137,350.3033,59.0137-350.3033,0.854806,0.751778,0.539172,0.650983,0.883973,0.128241
129629,59.0137,389.2739,59.0137-389.2739,0.823682,0.886260,0.723124,0.515230,0.917952,0.145148
...,...,...,...,...,...,...,...,...,...
130790,980.7146,981.7177,980.7146-981.7177,0.743341,0.954130,0.878971,0.816034,0.817595,0.070631
108711,981.5305,981.7177,981.5305-981.7177,0.571209,0.850899,0.670445,0.544705,0.731794,0.111292
131593,981.7177,996.7096,981.7177-996.7096,0.557537,0.784976,0.563671,0.732291,0.554250,0.099503
134700,985.5091,985.5490,985.5091-985.549,0.798737,0.577365,0.723628,0.875572,0.644783,0.106148


time: 47.1 ms (started: 2023-05-27 18:18:17 -05:00)


In [83]:
# Filter by std
df_edge_embeddings_join_filter_count_weight_std = df_edge_embeddings_join_filter_count_weight_std[df_edge_embeddings_join_filter_count_weight_std["std"] <= 0.3]
df_edge_embeddings_join_filter_count_weight_std.to_csv("{}/output_greedy/edges_filter_weight_std/greedy_{}_edge-filter-weight-std.csv".format(dir, group[0]), index=False)
df_edge_embeddings_join_filter_count_weight_std

,source,target,idx,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5,std
133290,59.0137,59.0291,59.0137-59.0291,0.954735,0.928018,0.841093,0.601605,0.970185,0.136282
131497,59.0137,349.2758,59.0137-349.2758,0.796859,0.788310,0.560861,0.551982,0.807207,0.118268
110528,59.0137,350.2793,59.0137-350.2793,0.807013,0.779306,0.576143,0.563560,0.808223,0.112406
132969,59.0137,350.3033,59.0137-350.3033,0.854806,0.751778,0.539172,0.650983,0.883973,0.128241
129629,59.0137,389.2739,59.0137-389.2739,0.823682,0.886260,0.723124,0.515230,0.917952,0.145148
...,...,...,...,...,...,...,...,...,...
130790,980.7146,981.7177,980.7146-981.7177,0.743341,0.954130,0.878971,0.816034,0.817595,0.070631
108711,981.5305,981.7177,981.5305-981.7177,0.571209,0.850899,0.670445,0.544705,0.731794,0.111292
131593,981.7177,996.7096,981.7177-996.7096,0.557537,0.784976,0.563671,0.732291,0.554250,0.099503
134700,985.5091,985.5490,985.5091-985.549,0.798737,0.577365,0.723628,0.875572,0.644783,0.106148


time: 1.17 s (started: 2023-05-27 18:18:17 -05:00)


In [84]:
df_edge_embeddings_join_filter_count_weight_std = pd.read_csv("{}/output_greedy/edges_filter_weight_std/greedy_{}_edge-filter-weight-std.csv".format(dir, group[0]))
df_edge_embeddings_join_filter_count_weight_std

,source,target,idx,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5,std
0,59.0137,59.0291,59.0137-59.0291,0.954735,0.928018,0.841093,0.601605,0.970185,0.136282
1,59.0137,349.2758,59.0137-349.2758,0.796859,0.788310,0.560861,0.551982,0.807207,0.118268
2,59.0137,350.2793,59.0137-350.2793,0.807013,0.779306,0.576143,0.563560,0.808223,0.112406
3,59.0137,350.3033,59.0137-350.3033,0.854806,0.751778,0.539172,0.650983,0.883973,0.128241
4,59.0137,389.2739,59.0137-389.2739,0.823682,0.886260,0.723124,0.515230,0.917952,0.145148
...,...,...,...,...,...,...,...,...,...
131535,980.7146,981.7177,980.7146-981.7177,0.743341,0.954130,0.878971,0.816034,0.817595,0.070631
131536,981.5305,981.7177,981.5305-981.7177,0.571209,0.850899,0.670445,0.544705,0.731794,0.111292
131537,981.7177,996.7096,981.7177-996.7096,0.557537,0.784976,0.563671,0.732291,0.554250,0.099503
131538,985.5091,985.5490,985.5091-985.549,0.798737,0.577365,0.723628,0.875572,0.644783,0.106148


time: 185 ms (started: 2023-05-27 18:18:18 -05:00)


In [85]:
# Average weight
df_edge_embeddings_join_filter_count_weight_std_avg = df_edge_embeddings_join_filter_count_weight_std.copy()
df_edge_embeddings_join_filter_count_weight_std_avg["weight"] = df_edge_embeddings_join_filter_count_weight_std_avg.iloc[:, -(len(subgroups) + 1):-1].mean(axis=1)
df_edge_embeddings_join_filter_count_weight_std_avg

,source,target,idx,subgroup1,subgroup2,subgroup3,subgroup4,subgroup5,std,weight
0,59.0137,59.0291,59.0137-59.0291,0.954735,0.928018,0.841093,0.601605,0.970185,0.136282,0.859127
1,59.0137,349.2758,59.0137-349.2758,0.796859,0.788310,0.560861,0.551982,0.807207,0.118268,0.701044
2,59.0137,350.2793,59.0137-350.2793,0.807013,0.779306,0.576143,0.563560,0.808223,0.112406,0.706849
3,59.0137,350.3033,59.0137-350.3033,0.854806,0.751778,0.539172,0.650983,0.883973,0.128241,0.736142
4,59.0137,389.2739,59.0137-389.2739,0.823682,0.886260,0.723124,0.515230,0.917952,0.145148,0.773249
...,...,...,...,...,...,...,...,...,...,...
131535,980.7146,981.7177,980.7146-981.7177,0.743341,0.954130,0.878971,0.816034,0.817595,0.070631,0.842014
131536,981.5305,981.7177,981.5305-981.7177,0.571209,0.850899,0.670445,0.544705,0.731794,0.111292,0.673811
131537,981.7177,996.7096,981.7177-996.7096,0.557537,0.784976,0.563671,0.732291,0.554250,0.099503,0.638545
131538,985.5091,985.5490,985.5091-985.549,0.798737,0.577365,0.723628,0.875572,0.644783,0.106148,0.724017


time: 36.7 ms (started: 2023-05-27 18:18:18 -05:00)


In [86]:
df_edge_embeddings_join_filter_count_weight_std_avg = df_edge_embeddings_join_filter_count_weight_std_avg.iloc[:, [0, 1, -1]]
df_edge_embeddings_join_filter_count_weight_std_avg.to_csv("{}/output_greedy/edges_filter_weight_std_avg/greedy_{}_edge-filter-weight-std-avg.csv".format(dir, group[0]), index=False)
df_edge_embeddings_join_filter_count_weight_std_avg

,source,target,weight
0,59.0137,59.0291,0.859127
1,59.0137,349.2758,0.701044
2,59.0137,350.2793,0.706849
3,59.0137,350.3033,0.736142
4,59.0137,389.2739,0.773249
...,...,...,...
131535,980.7146,981.7177,0.842014
131536,981.5305,981.7177,0.673811
131537,981.7177,996.7096,0.638545
131538,985.5091,985.5490,0.724017


time: 344 ms (started: 2023-05-27 18:18:19 -05:00)


In [87]:
df_edge_embeddings_join_filter_count_weight_std_avg = pd.read_csv("{}/output_greedy/edges_filter_weight_std_avg/greedy_{}_edge-filter-weight-std-avg.csv".format(dir, group[0]))
df_edge_embeddings_join_filter_count_weight_std_avg

,source,target,weight
0,59.0137,59.0291,0.859127
1,59.0137,349.2758,0.701044
2,59.0137,350.2793,0.706849
3,59.0137,350.3033,0.736142
4,59.0137,389.2739,0.773249
...,...,...,...
131535,980.7146,981.7177,0.842014
131536,981.5305,981.7177,0.673811
131537,981.7177,996.7096,0.638545
131538,985.5091,985.5490,0.724017


time: 49.6 ms (started: 2023-05-27 18:18:19 -05:00)


In [88]:
H2 = nx.from_pandas_edgelist(df_edge_embeddings_join_filter_count_weight_std_avg, "source", "target", edge_attr=["weight"])
graph_detail(H2)

list_graphs.append(H2.copy())

Num. nodes: 4181
Num. edges: 131540

time: 554 ms (started: 2023-05-27 18:18:19 -05:00)


### ANOVA

In [89]:
# Load dataset Groups
df1 = pd.read_csv("{}/input/Edwin_proyecto2/{}.csv".format(dir, "int1"), delimiter="|")
df2 = pd.read_csv("{}/input/Edwin_proyecto2/{}.csv".format(dir, "int2"), delimiter="|")
df3 = pd.read_csv("{}/input/Edwin_proyecto2/{}.csv".format(dir, "int3"), delimiter="|")
df4 = pd.read_csv("{}/input/Edwin_proyecto2/{}.csv".format(dir, "int4"), delimiter="|")
# df5_ = pd.read_csv("{}/inputs/Edwin_proyecto2/{}.csv".format(dir, "int5"), delimiter="|")

""" df1 = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities1"), delimiter="|")
df2 = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities2"), delimiter="|")
df3 = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities3"), delimiter="|")
df4 = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities4"), delimiter="|")
df5_ = pd.read_csv("{}/inputs/SecondDataset/{}.csv".format(dir, "intensities5"), delimiter="|") """

print(df1.shape)
print(df2.shape)
print(df3.shape)
print(df4.shape)
# print(df5_.shape)

(6245, 102)
(6245, 72)
(6245, 97)
(6245, 137)
time: 176 ms (started: 2023-05-27 18:18:20 -05:00)


In [90]:
# concat
# df_join_raw = pd.concat([df1.iloc[:,1:], df2.iloc[:, 2:], df3.iloc[:, 2:], df4.iloc[:, 2:], df5.iloc[:, 2:]], axis=1)
df_join_raw = pd.concat([df1.iloc[:, 1:], df2.iloc[:, 2:], df3.iloc[:, 2:], df4.iloc[:, 2:]], axis=1)
df_join_raw.set_index(["ionMz"], inplace=True)

print(df_join_raw.shape)
df_join_raw

(6245, 400)


,0001 / zwf1^ 3.4,0002 / zwf1^ 3.4,0003 / zwf1^ 3.4,0004 / zwf1^ 3.4,0005 / zwf1^ 3.4,0006 / zwf1^ 3.4,0007 / zwf1^ 3.4,0008 / zwf1^ 3.4,0009 / zwf1^ 3.4,0010 / zwf1^ 3.4,...,0951 / WT 3.4,0952 / WT 3.4,0953 / WT 3.4,0954 / WT 3.4,0955 / WT 3.4,0956 / WT 3.4,0957 / WT 3.4,0958 / WT 3.4,0959 / WT 3.4,0960 / WT 3.4
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,1,47,508,360,675,232,59,345,346,504,...,45,817,32,307,410,716,228,187,361,952
59.0137,53182,57902,51861,62812,54291,54395,59054,52547,57544,63538,...,45747,39677,47205,38697,54320,62610,47283,49927,43632,48511
59.0291,2151,2102,2334,2864,2393,2246,2331,1954,2310,2528,...,1852,1717,1830,1660,2006,2539,2103,2276,1643,2522
59.0370,83,174,366,1134,693,464,235,479,443,691,...,225,349,185,429,186,278,364,368,115,887
59.0453,1,1,51,642,493,143,11,227,160,154,...,60,293,1,336,65,142,1,182,10,684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,2470,2435,2465,2727,2773,2157,2168,2623,2256,2636,...,1866,1834,1669,674,2533,1017,2983,1814,2160,1899
996.7096,3366,3241,2909,3469,3538,3010,3031,2872,2709,3601,...,1748,1419,1160,621,1758,1474,1973,1023,1769,1629
997.5542,2818,2335,1807,1679,2222,2070,2158,1835,2087,2784,...,1168,828,1162,1103,2253,1419,2306,1542,827,1433


time: 23.4 ms (started: 2023-05-27 18:18:20 -05:00)


In [91]:
# get groud
if group[0] == "zwf1^":
    r = "zwf1"
elif group[0] == "pck1^":
    r = "pck1"
else:
    r = group[0]

df_raw_group = df_join_raw.filter(regex=r, axis=1)
df_raw_group

,0071 / WT 2.2,0072 / WT 2.2,0073 / WT 2.2,0074 / WT 2.2,0075 / WT 2.2,0076 / WT 2.2,0077 / WT 2.2,0078 / WT 2.2,0079 / WT 2.2,0080 / WT 2.2,...,0951 / WT 3.4,0952 / WT 3.4,0953 / WT 3.4,0954 / WT 3.4,0955 / WT 3.4,0956 / WT 3.4,0957 / WT 3.4,0958 / WT 3.4,0959 / WT 3.4,0960 / WT 3.4
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,528,1,378,438,530,250,662,431,305,334,...,45,817,32,307,410,716,228,187,361,952
59.0137,56523,24625,78586,76559,67449,74919,88152,99718,86001,93962,...,45747,39677,47205,38697,54320,62610,47283,49927,43632,48511
59.0291,2290,927,3149,3038,2774,3419,3911,3820,3374,3746,...,1852,1717,1830,1660,2006,2539,2103,2276,1643,2522
59.0370,320,195,1137,642,588,799,602,606,768,372,...,225,349,185,429,186,278,364,368,115,887
59.0453,196,59,555,405,234,413,46,148,95,25,...,60,293,1,336,65,142,1,182,10,684
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,2050,1415,3851,2285,2038,3089,2737,2788,1819,2600,...,1866,1834,1669,674,2533,1017,2983,1814,2160,1899
996.7096,2681,1984,3570,2169,2479,2668,3033,3271,2264,2411,...,1748,1419,1160,621,1758,1474,1973,1023,1769,1629
997.5542,1516,1307,2299,2363,2877,1436,1771,2326,1629,2740,...,1168,828,1162,1103,2253,1419,2306,1542,827,1433


time: 24.9 ms (started: 2023-05-27 18:18:20 -05:00)


In [92]:
# Logarithm

df_raw_log = df_raw_group.copy()
for column in df_raw_group.columns:
  df_raw_log[column] = np.log10(df_raw_group[column], where=df_raw_group[column]>0)
  # df_raw_log[column] = np.log10(df_raw_group[column], out=np.zeros_like(df_raw_group[column]), where=df_raw_group[column]>0)
df_raw_log

,0071 / WT 2.2,0072 / WT 2.2,0073 / WT 2.2,0074 / WT 2.2,0075 / WT 2.2,0076 / WT 2.2,0077 / WT 2.2,0078 / WT 2.2,0079 / WT 2.2,0080 / WT 2.2,...,0951 / WT 3.4,0952 / WT 3.4,0953 / WT 3.4,0954 / WT 3.4,0955 / WT 3.4,0956 / WT 3.4,0957 / WT 3.4,0958 / WT 3.4,0959 / WT 3.4,0960 / WT 3.4
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,2.722634,0.000000,2.577492,2.641474,2.724276,2.397940,2.820858,2.634477,2.484300,2.523746,...,1.653213,2.912222,1.505150,2.487138,2.612784e+00,2.854913e+00,2.357935,2.271842,2.557507,2.978637
59.0137,4.752225,4.391376,4.895345,4.883996,4.828976,4.874592,4.945232,4.998774,4.934504,4.972952,...,4.660363,4.598539,4.673988,4.587677,4.734960e+00,4.796644e+00,4.674705,4.698335,4.639805,4.685840
59.0291,3.359835,2.967080,3.498173,3.482588,3.443106,3.533899,3.592288,3.582063,3.528145,3.573568,...,3.267641,3.234770,3.262451,3.220108,3.302331e+00,3.404663e+00,3.322839,3.357172,3.215638,3.401745
59.0370,2.505150,2.290035,3.055760,2.807535,2.769377,2.902547,2.779596,2.782473,2.885361,2.570543,...,2.352183,2.542825,2.267172,2.632457,2.269513e+00,2.444045e+00,2.561101,2.565848,2.060698,2.947924
59.0453,2.292256,1.770852,2.744293,2.607455,2.369216,2.615950,1.662758,2.170262,1.977724,1.397940,...,1.778151,2.466868,0.000000,2.526339,1.812913e+00,2.152288e+00,0.000000,2.260071,1.000000,2.835056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,3.311754,3.150756,3.585574,3.358886,3.309204,3.489818,3.437275,3.445293,3.259833,3.414973,...,3.270912,3.263399,3.222456,2.828660,3.403635e+00,3.007321e+00,3.474653,3.258637,3.334454,3.278525
996.7096,3.428297,3.297542,3.552668,3.336260,3.394277,3.426186,3.481872,3.514681,3.354876,3.382197,...,3.242541,3.151982,3.064458,2.793092,3.245019e+00,3.168497e+00,3.295127,3.009876,3.247728,3.211921
997.5542,3.180699,3.116276,3.361539,3.373464,3.458940,3.157154,3.248219,3.366610,3.211921,3.437751,...,3.067443,2.918030,3.065206,3.042576,3.352761e+00,3.151982e+00,3.362859,3.188084,2.917506,3.156246


time: 104 ms (started: 2023-05-27 18:18:20 -05:00)


In [93]:
subgroups = [item.split("{} ".format(group[0]))[1].split(".")[0] for item in list(df_raw_log.columns)]
subgroups = np.unique(subgroups)
subgroups

array(['1', '2', '3', '4', '5'], dtype='<U1')

time: 3.41 ms (started: 2023-05-27 18:18:21 -05:00)


In [94]:
# Split graph

list_raw = []

for item in subgroups:
  list_raw.append(df_raw_log.filter(like="{} {}.".format(group[0], item)))

print(len(list_raw))
list_raw[0]

5


,0091 / WT 1.3,0092 / WT 1.3,0093 / WT 1.3,0094 / WT 1.3,0095 / WT 1.3,0096 / WT 1.3,0097 / WT 1.3,0098 / WT 1.3,0099 / WT 1.3,0100 / WT 1.3,...,0821 / WT 1.4,0822 / WT 1.4,0823 / WT 1.4,0824 / WT 1.4,0825 / WT 1.4,0826 / WT 1.4,0827 / WT 1.4,0828 / WT 1.4,0829 / WT 1.4,0830 / WT 1.4
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0049,2.515874,2.453318,2.832509,2.687529,2.515874,1.973128,2.873902,2.670246,2.795185,2.675778,...,2.686636,2.658011,2.689309e+00,2.902003e+00,2.872739,2.378398,2.842609,2.836957,3.044148,2.672098
59.0137,5.101637,5.073656,5.191384,5.053144,5.127630,5.014327,5.236834,5.009944,5.198135,5.278424,...,4.904088,4.937468,4.943069e+00,4.867391e+00,4.962649,4.961786,4.989499,4.914708,4.984023,4.986274
59.0291,3.683497,3.647872,3.772762,3.684486,3.743196,3.626853,3.847819,3.593618,3.759290,3.860038,...,3.530072,3.522053,3.543323e+00,3.539076e+00,3.593618,3.594614,3.639088,3.568671,3.661623,3.615529
59.0370,2.841985,2.990783,3.060320,3.089198,3.143951,2.880242,3.228657,2.829947,2.930949,3.070776,...,2.990339,3.022428,2.748963e+00,2.927883e+00,2.843233,2.973128,2.937518,3.072985,2.995196,2.896526
59.0453,0.000000,2.475671,2.632457,2.597695,2.862131,2.468347,2.568202,2.553883,2.403121,2.113943,...,2.499687,2.424882,0.000000e+00,2.588832e+00,2.459392,2.665581,2.596597,2.874482,2.693727,2.397940
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
996.5509,3.431203,3.400538,3.375481,3.477266,3.579669,3.106191,3.448397,3.398461,3.493179,3.498724,...,3.274620,3.240549,3.348500e+00,2.993436e+00,3.167022,3.196729,3.121560,3.302980,3.147985,3.387034
996.7096,2.877371,3.203305,3.030600,3.000000,3.068557,3.232996,2.970812,3.075547,3.157457,3.283075,...,2.978181,3.010724,3.106191e+00,2.926342e+00,2.813581,2.968016,2.629410,2.843855,3.015360,3.181272
997.5542,3.253822,3.159868,3.400538,3.302547,3.332842,3.306639,3.492201,3.340841,3.473633,3.412461,...,2.882525,3.237795,2.848189e+00,2.903090e+00,3.129045,2.842609,3.232742,3.332438,3.104828,2.862728


time: 36.6 ms (started: 2023-05-27 18:18:21 -05:00)


In [95]:
list_raw_copy = list_raw[:]

for k, item in enumerate(list_raw_copy):
    item.columns = [chr(65 + k)]*len(item.columns)

time: 1.98 ms (started: 2023-05-27 18:18:21 -05:00)


In [96]:
# Filter by graph and concat 
nodes = list(H2.nodes())
df_raw_filter = list_raw_copy[0].loc[nodes, :]

for k in range(1, len(subgroups)):
    df_temp = list_raw_copy[k].loc[nodes, :]
    # df_raw_filter = df_raw_filter.join(df_temp)
    df_raw_filter = pd.concat([df_raw_filter, df_temp], axis=1)

df_raw_filter.to_csv("{}/output_greedy/matrix/greedy_{}_matrix_copy.csv".format(dir, group[0]), index=True)
df_raw_filter

,A,A,A,A,A,A,A,A,A,A,...,E,E,E,E,E,E,E,E,E,E
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0137,5.101637,5.073656,5.191384,5.053144,5.127630,5.014327,5.236834,5.009944,5.198135,5.278424,...,4.675879,4.755280,4.687681,4.671071,4.633378,4.749458,4.704485,4.698370,4.683947,4.712448
59.0291,3.683497,3.647872,3.772762,3.684486,3.743196,3.626853,3.847819,3.593618,3.759290,3.860038,...,3.373831,3.409764,3.314710,3.294687,3.278754,3.404149,3.325516,3.282622,3.417804,3.319314
349.2758,5.257571,5.206707,5.276522,5.255923,5.214160,5.247069,5.299169,5.262506,5.377057,5.338753,...,4.657486,4.689522,4.707161,4.748095,4.676099,4.724030,4.739667,4.717254,4.714238,4.716429
350.2793,4.554683,4.540742,4.586981,4.581403,4.579544,4.554307,4.618613,4.589425,4.682118,4.677434,...,4.039969,4.066214,4.061867,4.025961,3.994625,4.081167,4.060131,4.048985,4.002986,4.008387
350.3033,3.862847,3.763727,3.853272,3.801129,3.848128,3.760799,3.868879,3.779957,3.918816,3.860098,...,3.320562,3.282849,3.381837,3.264109,3.362294,3.343802,3.442950,3.299943,3.292034,3.311966
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980.0986,2.822822,2.903633,2.933487,3.033424,2.773786,2.898176,3.129690,2.778874,2.737987,2.861534,...,3.125156,2.392697,2.866878,3.100026,2.763428,2.744293,2.962369,2.707570,3.104146,3.174641
985.5091,3.099335,3.335658,3.382197,3.236789,3.277151,3.104146,3.169674,3.337260,3.256718,3.251395,...,3.197005,3.309204,3.135451,3.409933,3.204391,3.344392,3.358506,3.187803,3.273927,3.288026
985.5490,3.133858,3.294687,3.276692,3.133219,3.239299,3.212720,2.837588,3.193681,3.090611,3.307068,...,3.436957,3.440437,3.449015,3.569608,3.458336,3.533391,3.477555,3.477121,3.437592,3.498724


time: 955 ms (started: 2023-05-27 18:18:21 -05:00)


In [97]:
# df_raw_filter = pd.read_csv("{}/output_greedy/matrix/greedy_{}_matrix_copy.csv".format(dir, group[0]), index_col=0n)
# df_raw_filter

time: 473 µs (started: 2023-05-27 18:18:22 -05:00)


---

In [98]:
""" df_raw_filter = pd.read_csv("/home/ealvarez/Project/GNN_Filter/GNN_unsupervised/luis.csv")
df_raw_filter

df_raw_filter.columns = ["id", "A", "A", "B", "B"]
# df_raw_filter.drop("id", axis=1, inplace=True)
df_raw_filter.set_index("id", inplace=True)
df_raw_filter

p_values = anova(df_raw_filter)

df_raw_filter["p-value"] = p_values
df_raw_filter """

' df_raw_filter = pd.read_csv("/home/ealvarez/Project/GNN_Filter/GNN_unsupervised/luis.csv")\ndf_raw_filter\n\ndf_raw_filter.columns = ["id", "A", "A", "B", "B"]\n# df_raw_filter.drop("id", axis=1, inplace=True)\ndf_raw_filter.set_index("id", inplace=True)\ndf_raw_filter\n\np_values = anova(df_raw_filter)\n\ndf_raw_filter["p-value"] = p_values\ndf_raw_filter '

time: 3.81 ms (started: 2023-05-27 18:18:22 -05:00)


In [99]:
# df_raw_filter_p = df_raw_filter[df_raw_filter["p-value"] < 0.001]
# df_raw_filter_p

time: 421 µs (started: 2023-05-27 18:18:22 -05:00)


---
---

In [100]:
# ANOVA
df_raw_filter_anova = df_raw_filter.copy()
p_values = anova(df_raw_filter_anova)
df_raw_filter_anova["p-value"] = p_values
df_raw_filter_anova

,A,A,A,A,A,A,A,A,A,A,...,E,E,E,E,E,E,E,E,E,p-value
ionMz,,,,,,,,,,,,,,,,,,,,,
59.0137,5.101637,5.073656,5.191384,5.053144,5.127630,5.014327,5.236834,5.009944,5.198135,5.278424,...,4.755280,4.687681,4.671071,4.633378,4.749458,4.704485,4.698370,4.683947,4.712448,7.850264e-16
59.0291,3.683497,3.647872,3.772762,3.684486,3.743196,3.626853,3.847819,3.593618,3.759290,3.860038,...,3.409764,3.314710,3.294687,3.278754,3.404149,3.325516,3.282622,3.417804,3.319314,9.520388e-15
349.2758,5.257571,5.206707,5.276522,5.255923,5.214160,5.247069,5.299169,5.262506,5.377057,5.338753,...,4.689522,4.707161,4.748095,4.676099,4.724030,4.739667,4.717254,4.714238,4.716429,2.211955e-11
350.2793,4.554683,4.540742,4.586981,4.581403,4.579544,4.554307,4.618613,4.589425,4.682118,4.677434,...,4.066214,4.061867,4.025961,3.994625,4.081167,4.060131,4.048985,4.002986,4.008387,3.759874e-11
350.3033,3.862847,3.763727,3.853272,3.801129,3.848128,3.760799,3.868879,3.779957,3.918816,3.860098,...,3.282849,3.381837,3.264109,3.362294,3.343802,3.442950,3.299943,3.292034,3.311966,1.789076e-06
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
980.0986,2.822822,2.903633,2.933487,3.033424,2.773786,2.898176,3.129690,2.778874,2.737987,2.861534,...,2.392697,2.866878,3.100026,2.763428,2.744293,2.962369,2.707570,3.104146,3.174641,7.257253e-01
985.5091,3.099335,3.335658,3.382197,3.236789,3.277151,3.104146,3.169674,3.337260,3.256718,3.251395,...,3.309204,3.135451,3.409933,3.204391,3.344392,3.358506,3.187803,3.273927,3.288026,1.744356e-01
985.5490,3.133858,3.294687,3.276692,3.133219,3.239299,3.212720,2.837588,3.193681,3.090611,3.307068,...,3.440437,3.449015,3.569608,3.458336,3.533391,3.477555,3.477121,3.437592,3.498724,4.576667e-05


time: 15 s (started: 2023-05-27 18:18:22 -05:00)


In [101]:
# Filter by ANOVA (p > 0.001)
df_raw_filter_anova_p = df_raw_filter_anova[df_raw_filter_anova["p-value"] > 0.001]
df_raw_filter_anova_p

nodes = list(df_raw_filter_anova_p.index)
len(nodes)

1651

time: 7.09 ms (started: 2023-05-27 18:18:38 -05:00)


#### Filter graph by ANOVA

In [108]:
H3 = H2.subgraph(nodes)
graph_detail(H3)

list_graphs.append(H3.copy())

Num. nodes: 1651
Num. edges: 11868

time: 103 ms (started: 2023-05-27 18:25:38 -05:00)


In [103]:
edges = list(H3.edges())

df_edge_embeddings_join_filter_count_weight_std_avg_anova = pd.DataFrame(edges, columns=["source", "target"])
df_edge_embeddings_join_filter_count_weight_std_avg_anova["weight"] = [H3.get_edge_data(edge[0], edge[1], default=0)["weight"] for edge in edges]
df_edge_embeddings_join_filter_count_weight_std_avg_anova.to_csv("{}/output_greedy/edges_filter_weight_std_avg_anova/greedy_{}_edge-filter-weight-std-avg-anova.csv".format(dir, group[0]), index=False)
df_edge_embeddings_join_filter_count_weight_std_avg_anova

,source,target,weight
0,61.9884,293.1786,0.596832
1,122.9910,123.0007,0.989721
2,122.9910,123.0188,0.969324
3,122.9910,123.0360,0.900589
4,122.9910,123.9906,0.952875
...,...,...,...
11863,492.9771,493.0401,0.805087
11864,494.9997,495.0368,0.774382
11865,504.1338,504.0519,0.676079
11866,505.1609,505.1361,0.764508


time: 143 ms (started: 2023-05-27 18:18:38 -05:00)


In [104]:
df_edge_embeddings_join_filter_count_weight_std_avg_anova = pd.read_csv("{}/output_greedy/edges_filter_weight_std_avg_anova/greedy_{}_edge-filter-weight-std-avg-anova.csv".format(dir, group[0]))
df_edge_embeddings_join_filter_count_weight_std_avg_anova

,source,target,weight
0,61.9884,293.1786,0.596832
1,122.9910,123.0007,0.989721
2,122.9910,123.0188,0.969324
3,122.9910,123.0360,0.900589
4,122.9910,123.9906,0.952875
...,...,...,...
11863,492.9771,493.0401,0.805087
11864,494.9997,495.0368,0.774382
11865,504.1338,504.0519,0.676079
11866,505.1609,505.1361,0.764508


time: 21.8 ms (started: 2023-05-27 18:18:38 -05:00)


In [105]:
H4 = nx.from_pandas_edgelist(df_edge_embeddings_join_filter_count_weight_std_avg_anova, "source", "target")
graph_detail(H4)

list_graphs.append(H4.copy())

Num. nodes: 1452
Num. edges: 11868

time: 52.8 ms (started: 2023-05-27 18:18:38 -05:00)


In [106]:
for graph in list_graphs:
    graph_detail(graph)
    print()

Num. nodes: 4202
Num. edges: 135076


Num. nodes: 4181
Num. edges: 131540


Num. nodes: 1651
Num. edges: 11868


Num. nodes: 1452
Num. edges: 11868


time: 11.9 ms (started: 2023-05-27 18:18:38 -05:00)
